In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
#     url = 'https://api.s0.os.hmny.io/'
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [7]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [8]:
def getBlockSigner(block):
    method = "hmyv2_getBlockSigners"
    params = [block]
    return get_information(method, params)['result']

In [9]:
def getBlockByNumber(block):
    method = "hmy_getBlockByNumberNew"
    params = [hex(block),
             {"withSigners":True}]
    return get_information(method, params)

In [10]:
def CN1_test(single):
    logger.info(f"Test-CN1: Slow validator is never starved (should be able to sign blocks)")
    curr_test = None
    
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the 5th block in the new epoch")
        while block < last_block+5:
            block = getBlockNumber()
    logger.info(f"current block: {block}, will begin collecting infos...")
    signer_address = getBlockSigner(block)
    acc_rewards_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        if address in signer_address:
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated

    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    # get the validator info and compute validator rewards
    new_signer_address = getBlockSigner(block)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        if address in new_signer_address:           
            reward_accumulated = i['lifetime']['reward-accumulated']
            if address not in acc_rewards_prev:
                continue
            reward = reward_accumulated - acc_rewards_prev[address]
            if reward == 0:
                flag = False
                logger.warning(f"Test-CN1: Fail")
                logger.warning(f"validator {address} who is a signer doesn't get reward\n")
    if flag:
        logger.info(f"Test-CN1: All signers get rewards")
    
    while block < last_block:
        block = getBlockNumber()
    logger.info(f"last block in this epoch reached, {block}")
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['current-epoch-performance']:
            sign = i['current-epoch-performance']['current-epoch-signing-percent']
            if sign['current-epoch-to-sign'] == 0:
                continue
            perc = float(sign['current-epoch-signing-percentage'])
            if perc > 2/3:
                address = i['validator']['address']
                epos_status = i['epos-status']
                if epos_status == 'not eligible to be elected next epoch':
                    flag = False
                    logger.warning(f"Test-CN1: Fail")
                    logger.warning(f"validator {address} who is a signer is not eligible to be elected next epoch\n")

    if flag:
        logger.info(f"Test-CN1: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [12]:
CN1_test(True)

INFO:economic-test:Test-CN1: Slow validator is never starved (should be able to sign blocks)
INFO:economic-test:current and last block numbers: 3417961, 3424255
INFO:economic-test:current block: 3417961, will begin collecting infos...


current and last block numbers 3417961 3424255


INFO:economic-test:new block 3417962 reached, will begin testing...
INFO:economic-test:Test-CN1: All signers get rewards
INFO:economic-test:last block in this epoch reached, 3424255
INFO:economic-test:Test-CN1: Succeed



(True, None)

In [ ]:
# def getValidatorInfo(validator):
#     method = "hmy_getValidatorInformation"
#     params = [validator]
#     return get_information(method, params)['result']

In [ ]:
# i = 'one103q7qe5t2505lypvltkqtddaef5tzfxwsse4z7'
# getValidatorInfo(i)